Atividades
===

Essas atividade são relativas a aulas de extração de atributos e selecão de atributos. 
- Link para video: https://drive.google.com/file/d/13LLQGLt7QsjKshBXgUdJ4RVOT8N252cU/view
- Link para apresentação: https://docs.google.com/presentation/d/1wctFgQe7TSlBEypZbVQqqsfrnpSgOeXPSO-mCJ6jIz4


> Lembre de criar uma `virtualenv` com os `requirements.txt` do repositório.

> Lembre de criar uma `kernel` do jupyter para seus desenvolvimentos

> **MANTENHA** essa arquivo no local padrão do repositório. Detro da pasta **feature_extraction_and_feature_selection**.

> Enviei apenas o seu notebook para correção. Não é necessário enviar nenhum outro arquivo. Siga o padrão `Atividades-<NOME-DO-ALUNO>.ipynb`

# Feature Extraction

## 1. Realizar a extração de atributos utilzando as técnicas Fourier e HOS nas bases de dados

- Dados de falhas em aerogeradores

As orientações abaixo devem ser seguidas antes de iniciar os trabalhos

> Todo os arquivos nomeados com `v000_ ... .csv` são referentes a essa base.

> A classe referente ao dado está representada por uma TAG no nome do arquivo da seguinte forma: `v000_NORMAL_ ... .csv` é o dado referente a classe de funcionamento normal do gerador. A TAG `v000_SC_LI_LVL3_ ... .csv` é a classe referente a falha tipo 1 e continua. Não se preocupem com a ordem de enumeração das classes nesse momento.

> Há 5 arquivos na pasta.

> Utilizar as colunas a `Current_R`, `Current_S`, `Current_T` para realizar extração de atributos. Perceba que cada coluna é referente a uma sinal de corrente elétrica, portanto cada um são representados vetores de dimensões $1x50000$.

> Antes de realizar a extração de atributos deve-se dividir cada sinal em 10 partes de tamanho $1x5000$. A ideia aqui é aumentar o número de amostra da base de dados por 10. Pense que cada parte é um recorte do sinal, e ao serem concatenadas retornarão o sinal original.

# Imports

In [308]:
import pandas as pd
import numpy as np
from os import path
from feature_extraction_signal.src import feature_extraction
from feature_selection_framework.src.feature_selection import*

### Carregue a base

In [39]:
DATAFOLDER = path.join('..', '..', 'data')
DATAFILES = [
        'v000_FAULT_SC_HI_LVL2_FR6000_FG5942_L000_0,6IN_SENSORC.csv', 
        'v000_FAULT_SC_LI_LVL3_FR4500_FG4365_L000_0,8IN_SENSORC.csv',
        'v000_FAULT_SC_LI_LVL3_FR6000_FG5927_L000_0,4IN_SENSORC.csv',
        'v000_NORMAL_FR4500_FG4385_L000_1,0IN_SENSORC.csv',
        'v000_NORMAL_FR6000_FG5955_L000_0,5IN_SENSORC.csv']

SEED = 1987298712

data_normal = pd.read_csv(path.join(DATAFOLDER, DATAFILES[4]))
data_fault = pd.read_csv(path.join(DATAFOLDER, DATAFILES[2]))

### a) Comparativo da média do sinal e sua distribuição

- Realize o calculo da média e desvio padrão em cada um dos sinais e exiba o resultado em um dataframe. Discuta suas conclusões.

**help**: utilize as funcões nativas do numpy ou do pandas. Lembre que o sinal é uma senoide, qual a média em uma senoide simétrica?

In [40]:
media_list=[]
desvio_list=[]

for i in range(14):    
    media_list.append(data_normal.mean().values[i])
    desvio_list.append(data_normal.std().values[i])
    
df_media = pd.DataFrame(data=media_list, columns=['Média'])
df_desvio = pd.DataFrame(data=desvio_list, columns=['Desvio'])

pd.concat([df_media, df_desvio], axis=1, join='inner')

,Média,Desvio
0,-0.010286,1.184251e+00
1,0.012611,1.188874e+00
2,0.002357,1.197354e+00
3,2.499768,1.355010e-01
4,2.496935,1.350311e-01
5,2.491380,1.254342e-01
6,60.000000,0.000000e+00
7,59.550000,3.192500e-11
8,210.000000,0.000000e+00
9,0.100000,1.317015e-14


### b) Extração de atributos no sinal

- Utilize o código de `feature_extraction_signal` em: https://github.com/navarmn/feature_extraction_signal
- Use as classes `Fourier` e `HOS` de maneira adequada. Lembre do método `.transform()`.
- Na classe `Fourier`utilize os parâmetros: `fs=5000`; o valor de fundamental está na TAG do nome em `_FG5955_`significa `fundamental=59.55`. `harmonics=(0.5, 1, 1.5, 2.5, 3, 5, 7)`, mas sintam-se à vontade para buscar outras harmônicas;
- Os rótulos devem serem impostos de acordo com a descrição da tag feita cima. 
- Monte um dataframe que contenham os atributos em cada coluna e a útlima com o rótulo. Concatene os artibutos extraídos da `Current_R`, `Current_S` e `Current_T`.
- Deverá ser feito um dataframe para os atributos de Fourier e um para os atributos de HOS.

In [41]:
fe_fourier = feature_extraction.Fourier(fundamental=59.55, fs=5000, harmonics=(0.5, 1, 1.5, 2.5, 3, 5, 7))

In [42]:
fourierR=[]
fourierS=[]
fourierT=[]

for datafile in DATAFILES:
    data = pd.read_csv(path.join(DATAFOLDER, datafile))
    
    vetorFR = fe_fourier.transform(data['Current_R'])
    fourierR.append(vetorFR['features']) 
    vetorFS = fe_fourier.transform(data['Current_S'])
    fourierS.append(vetorFS['features'])
    vetorFT = fe_fourier.transform(data['Current_T'])
    fourierT.append(vetorFT['features'])

df_FR = pd.DataFrame(data=fourierR, columns= ['0.5', '1', '1.5', '2.5', '3', '5', '7'])
df_FS = pd.DataFrame(data=fourierS, columns= ['0.5', '1', '1.5', '2.5', '3', '5', '7'])
df_FT = pd.DataFrame(data=fourierT, columns= ['0.5', '1', '1.5', '2.5', '3', '5', '7'])


A=df_FR.rename(columns={'0.5': '0.5_R', '1': '1_R','1.5': '1.5_R', '2.5': '2.5_R', '3': '3_R', '5': '5_R', '7': '7_R'})
B=df_FS.rename(columns={'0.5': '0.5_S', '1': '1_S','1.5': '1.5_S', '2.5': '2.5_S', '3': '3_S', '5': '5_S', '7': '7_S'})
C=df_FT.rename(columns={'0.5': '0.5_T', '1': '1_T','1.5': '1.5_T', '2.5': '2.5_T', '3': '3_T', '5': '5_T', '7': '7_T'})

In [43]:
Fourier = pd.concat([A,B,C],axis=1, join='inner')
Fourier

,0.5_R,1_R,1.5_R,2.5_R,3_R,5_R,7_R,0.5_S,1_S,1.5_S,...,3_S,5_S,7_S,0.5_T,1_T,1.5_T,2.5_T,3_T,5_T,7_T
0,0.001095,0.001277,0.001918,0.002628,0.002227,0.002919,0.002225,0.001408,0.002476,0.002738,...,0.001564,0.003230,0.002526,0.002274,0.004525,0.002392,0.002334,0.002347,0.002364,0.002707
1,0.002822,0.003791,0.019491,0.005401,0.003715,0.004000,0.004861,0.002159,0.002268,0.020921,...,0.005281,0.007329,0.005246,0.004543,0.007272,0.021583,0.008734,0.008741,0.008049,0.006918
2,0.001429,0.004262,0.003552,0.003507,0.004367,0.003292,0.003073,0.001102,0.004450,0.003826,...,0.002369,0.003196,0.001750,0.001230,0.003121,0.002720,0.003168,0.002555,0.001651,0.001859
3,0.005041,0.005721,0.017829,0.010172,0.008299,0.010946,0.007722,0.004242,0.004446,0.013907,...,0.007882,0.007870,0.006494,0.002791,0.003975,0.018066,0.008152,0.006263,0.005550,0.004520
4,0.001962,0.002993,0.002656,0.003446,0.002962,0.003507,0.003205,0.003298,0.006923,0.005191,...,0.003641,0.002768,0.003440,0.002696,0.005173,0.004397,0.003026,0.002255,0.003243,0.003083


In [44]:
fe_hosR = feature_extraction.HOS()
fe_hosR.transform(data_normal['Current_R'])
fe_hosS = feature_extraction.HOS()
fe_hosS.transform(data_normal['Current_S'])
fe_hosT = feature_extraction.HOS()
fe_hosT.transform(data_normal['Current_T'])

hos_listR = []
hos_listS = []
hos_listT = []

for datafile in DATAFILES:
    data = pd.read_csv(path.join(DATAFOLDER, datafile))
    feature_vector_hosR = fe_hosR.transform(data['Current_R'])
    hos_listR.append(feature_vector_hosR['features'])
    feature_vector_hosS = fe_hosS.transform(data['Current_S'])
    hos_listS.append(feature_vector_hosS['features'])
    feature_vector_hosT = fe_hosT.transform(data['Current_T'])
    hos_listT.append(feature_vector_hosT['features'])
    
df_hosR = pd.DataFrame(data=hos_listR, columns=['Rms', 'Skewness', 'Kurtosis', 'Variance'])
df_hosS = pd.DataFrame(data=hos_listS, columns=['Rms', 'Skewness', 'Kurtosis', 'Variance'])
df_hosT = pd.DataFrame(data=hos_listT, columns=['Rms', 'Skewness', 'Kurtosis', 'Variance'])


D=df_hosR.rename(columns={'Rms': 'Rms_R', 'Skewness': 'Skewness_R','Kurtosis': 'Kurtosis_R', 'Variance': 'Variance_R'})
E=df_hosS.rename(columns={'Rms': 'Rms_S', 'Skewness': 'Skewness_S','Kurtosis': 'Kurtosis_S', 'Variance': 'Variance_S'})
F=df_hosT.rename(columns={'Rms': 'Rms_T', 'Skewness': 'Skewness_T','Kurtosis': 'Kurtosis_T', 'Variance': 'Variance_T'})



HOS = pd.concat([D,E,F], axis=1, join='inner')
HOS

,Rms_R,Skewness_R,Kurtosis_R,Variance_R,Rms_S,Skewness_S,Kurtosis_S,Variance_S,Rms_T,Skewness_T,Kurtosis_T,Variance_T
0,1.295173,1.677402,0.003296,-1.442273,1.211041,1.466524,0.006055,-1.436841,1.249027,1.560100,0.003729,-1.439101
1,3.148889,9.915628,0.000336,-1.520664,2.414731,5.830991,-0.008669,-1.326985,2.460111,6.052261,-0.005396,-1.443063
2,1.560922,2.436410,0.001083,-1.482916,1.293480,1.672948,0.006783,-1.361676,1.110920,1.234167,-0.002570,-1.407382
3,2.906343,8.446975,-0.005243,-1.472242,2.920356,8.528573,-0.004661,-1.473669,2.986445,8.919035,-0.005999,-1.442528
4,1.184284,1.402451,0.005195,-1.446144,1.188929,1.413422,0.001404,-1.448900,1.197345,1.433657,0.000487,-1.443700


In [45]:
Concatenar = pd.concat([Fourier, HOS], axis=1, join='inner')
Concatenar

,0.5_R,1_R,1.5_R,2.5_R,3_R,5_R,7_R,0.5_S,1_S,1.5_S,...,Kurtosis_R,Variance_R,Rms_S,Skewness_S,Kurtosis_S,Variance_S,Rms_T,Skewness_T,Kurtosis_T,Variance_T
0,0.001095,0.001277,0.001918,0.002628,0.002227,0.002919,0.002225,0.001408,0.002476,0.002738,...,0.003296,-1.442273,1.211041,1.466524,0.006055,-1.436841,1.249027,1.560100,0.003729,-1.439101
1,0.002822,0.003791,0.019491,0.005401,0.003715,0.004000,0.004861,0.002159,0.002268,0.020921,...,0.000336,-1.520664,2.414731,5.830991,-0.008669,-1.326985,2.460111,6.052261,-0.005396,-1.443063
2,0.001429,0.004262,0.003552,0.003507,0.004367,0.003292,0.003073,0.001102,0.004450,0.003826,...,0.001083,-1.482916,1.293480,1.672948,0.006783,-1.361676,1.110920,1.234167,-0.002570,-1.407382
3,0.005041,0.005721,0.017829,0.010172,0.008299,0.010946,0.007722,0.004242,0.004446,0.013907,...,-0.005243,-1.472242,2.920356,8.528573,-0.004661,-1.473669,2.986445,8.919035,-0.005999,-1.442528
4,0.001962,0.002993,0.002656,0.003446,0.002962,0.003507,0.003205,0.003298,0.006923,0.005191,...,0.005195,-1.446144,1.188929,1.413422,0.001404,-1.448900,1.197345,1.433657,0.000487,-1.443700


### c) Análise exploratória na base criada com os extratores de atributos
- Use a ferramenta `pandas-profiling` https://github.com/pandas-profiling/pandas-profiling
- Faça um resumo do relatório, destacando os principais pontos gerados na base.
- Faça uma análise para cada uma das bases:
    1. Features criada com Fourier
    2. Features criada com HOS
    3. Combinação das duas features (Fourier + HOS). 
> Dica: dê atenção ao coeficiente de correlação.

In [10]:
from pandas_profiling import ProfileReport

profileF = ProfileReport(Fourier, title='Base Fourier', html={'style':{'full_width':True}})

In [11]:
profileF.to_widgets()

# Relatório da base Fourier

- No início do relatório, podemos ter uma visão geral e resumida do que está presente na base. Nessa base, existe 21 variáveis, nenhuma informação faltando e o tamanho da memória que ocupa no programa.
- Cada variável terá a média, mínimo e máximo, além de qual variável tem uma alta correlação.
- Toda a interação com si mesmo, é uma representação linear, mas a interação com as outras variáveis é variada, porque apresentação uma diferença grande de um ponto ao outro.
- A correlação de Pearson calcula o quanto uma variável depende da outra na base. É fácil ver que a corrente R tem uma grande correlação com as harmônicas dela, mas também com as correntes S e T, mesmo que a correlação seja um pouco menor. No entanto, a harmônica 1.0 das correntes S e T tem uma baixa e negativa correlação com as demais harmônicas. 

In [12]:
from pandas_profiling import ProfileReport

profileH = ProfileReport(HOS, title='Base HOS', html={'style':{'full_width':True}})

In [13]:
profileH.to_widgets()

# Relatório da base HOS

- No início temos uma visão geral na qual sabemos que na base existe 12 variáveis, nenhuma informação faltando e o total de memória ocupada na base.
- As variáveis sempre apresentam a média, mínimo e máximo dos dados, além de uma lista de variáveis que tem uma alta correlação com a variável desejada. 
- Quando estamos na janela de interação confirmamos quais as variáveis tem uma interação e correlação mostrada em cada variável, assim, quando os pontos estão alinhados, significa que tem uma alta interação, mas quanto mais disperso, a interação é menor.
- A correlação de rms com skewness ou vice-versa, de qualquer corrente é alta, mas a correlação de kurtosis e variânce tem uma correlação negativa com as demais variáveis, mas entre elas tem uma boa correlação. Já a variância da corrente S não há correlação com as outras variáveis.

In [9]:
from pandas_profiling import ProfileReport

profileFH = ProfileReport(Concatenar, title='Base Concatenada', html={'style':{'full_width':True}})

In [10]:
profileFH.to_widgets()

# Relatório das duas bases juntas

- Existem 33 variáveis a serem analisadas, nenhuma informação da nova base faltando e um total de memória utilizada. As informações das variáveis permanecem a mesma dos dois últimos relatórios, como média, máximo e mínimo.
- A interação das variáveis ainda permanecem as mesmas podendo ratificar as correlações.
- A correlação de Fourier e de HOS ainda permanecem como na representação anterior. As novas correlações entre as variáveis das duas bases, temos que as correntes R, S e T apresentam uma grande correlação com o rms e o skewness de todas as correntes.

# Feature Selection

## 1. Realizar um estudo de relevância de atributos utilzando as técnicas exibidas na aula.

Nesse momento utilize a base de dados `Clinical_data_09-09-19-processed.csv`. Uma breve descrição da base:
- Contém um histórico dos registros de pacientes acometidos com uma determinada patologia.
- Os registros são variados, vão desde resposta de questionários médicos de anamnese, por exemplo, "*É fumante nos últimos 5 anos?*", "*Faz uso de álcool constantemente?*", até resultados de exames clínicos como ECG e Ecocardiograma.
- O rótulo dessa base é a coluna `Óbito`.

### Carregue a base

In [196]:
DATAFOLDER = path.join('..', '..', 'data')
DATAFILES = [
        'Clinical_data_09-09-19-processed.csv'
]

ELETROCARDIOGRAMA = ['ECG ', 'FC',
       'Alt Prim', 'Dist Cond InterVent ', 'Dist Cond AtrioVent ',
       'Pausa > 3s ', 'ESV', 'EV', 'TVMNS', 'Area Elet inativa'] 

HISTORIA_PREVIA = [
       'Cancer', 'HAS', 'DM2', 'Cardiopatia Outra',
       'Marcapasso', 'Sincope', 'Fibrilação/Flutter Atrial', 'I R Crônica',
       'DLP', 'Coronariopatia', 'Embolia Pulmonar', 'Ins Cardiaca ', 'AVC',
       'DVP', 'TSH', 'Tabagismo', 'Alcoolismo', 'Sedentarismo']

data = pd.read_csv(path.join(DATAFOLDER, DATAFILES[0]))

In [197]:
X = data[HISTORIA_PREVIA]
y = data["Obito"]

### a) Análise exploratória na base de dados
- Use a ferramenta `pandas-profiling` https://github.com/pandas-profiling/pandas-profiling
- Faça um resumo do relatório, destacando os principais pontos gerados na base.

In [4]:
from pandas_profiling import ProfileReport

profile = ProfileReport(data, title='Seleção de Atributos')

In [5]:
profile.to_widgets()

# Relatório sobre a base

- Quando o relatório é gerado, podemos analisar de primeira que existem 60 (sessenta) variáveis e 410 informações faltando no documento analisado. Além da análise individual de cada variável que contém diferentes formas de atribuições, sendo algumas delas por nome, idade, zeros e uns.
- Na planilha de interações, o comportamento de cada variável individualmente é linear, mas a interação com outras variáveis varia, sendo que em algumas permanecem constantes ou com uma  representação independente podendo ter pontos acumulados em uma determinada região e outros dispersos pelo gráfico.
- A correlação de Pearson é uma medida de associação linear entre as variáveis. Se analisado o Óbito dos pacientes com o histórico de doenças, podemos identificar quais foram as possíveis causas do Óbito, onde estiver mais avermelhado possue uma correlação alta e quanto mais azul, existe uma correlação negativa.

### b) Identicar qual a variáveis parecem influenciar no óbito dos pacientes :(
- Utilize o código `feature_selection_framework` https://github.com/navarmn/feature_selection_framework 
- Monte o resultado em um `DataFrame`, as colunas serão referentes aos métodos e as linhas deverão ser os atributos. Coloque 1 quando o método indicar como relevante e 0 quando o método indicar com não relevante;
- Utilize o seguinte critério para validar um atributo como relevante: moda >= 3

In [198]:
colunas = ['Correlation', 'Backward Elimination', 'Recursive Feature Elimination','ANOVA', 'Embedded', 'Ensemble']

df_final = pd.DataFrame(index = HISTORIA_PREVIA, columns = colunas)

In [199]:
# Correlation
Cor = []
fs = Correlation()
fs.fit(X, y)
Cor = fs.relevant_features
df_Cor = pd.DataFrame(data=Cor, columns=['Correlation'])

# Backward Elimination (BE)
BE = []
fs1 = BackwardElimination()
fs1.fit(X, y)
BE = fs1.relevant_features
df_BE = pd.DataFrame(data = BE, columns = ['Backward Elimination']) 

# Recursive Feature Elimination (RFE)
RFE = []
fs2 = RecursiveFeatureElimination()
fs2.fit(X, y)
RFE = fs2.relevant_features
df_RFE = pd.DataFrame(data = RFE, columns = ['Recursive Feature Elimination'])

# ANOVA
Anova = []
fs3 = ANOVA()
fs3.fit(X, y)
Anova = fs3.relevant_features
df_Anova = pd.DataFrame(data = Anova, columns = ['ANOVA'])

# Embedded
Emb = []
fs4 = Embedded()
fs4.fit(X, y)
Emb = fs4.relevant_features
df_Emb = pd.DataFrame(data = Emb, columns = ['Embedded'])

# Ensemble
Ens = []
fs5 = Ensemble()
fs5.fit(X, y)
Ens = fs5.relevant_features
df_Ens = pd.DataFrame(data = Ens, columns = ['Ensemble'])

Optimum number of features: 10
Score with 10 features: 0.057256
Best alpha using built-in LassoCV: 0.013854
Best score using built-in LassoCV: 0.067040
Lasso picked 3 variables and eliminated the other 15 variables


In [258]:
cont = 0
for r in range(df_Cor.shape[0]):
    for hp in HISTORIA_PREVIA:
        if df_Cor.loc[r,'Correlation'] == hp:
            df_final.loc[hp,'Correlation'] = hp
            cont+=1
        else:
            cont+=1

cont = 0
for r in range(df_BE.shape[0]):
    for hp in HISTORIA_PREVIA:
        if df_BE.loc[r,'Backward Elimination'] == hp:
            df_final.loc[hp,'Backward Elimination'] = hp
            cont+=1
        else:
            cont+=1
            
cont = 0
for r in range(df_RFE.shape[0]):
    for hp in HISTORIA_PREVIA:
        if df_RFE.loc[r,'Recursive Feature Elimination'] == hp:
            df_final.loc[hp,'Recursive Feature Elimination'] = hp
            cont+=1
        else:
            cont+=1

cont = 0
for r in range(df_Anova.shape[0]):
    for hp in HISTORIA_PREVIA:
        if df_Anova.loc[r,'ANOVA'] == hp:
            df_final.loc[hp,'ANOVA'] = hp
            cont+=1
        else:
            cont+=1

cont = 0
for r in range(df_Emb.shape[0]):
    for hp in HISTORIA_PREVIA:
        if df_Emb.loc[r,'Embedded'] == hp:
            df_final.loc[hp,'Embedded'] = hp
            cont+=1
        else:
            cont+=1
        
cont = 0
for r in range(df_Ens.shape[0]):
    for hp in HISTORIA_PREVIA:
        if df_Ens.loc[r,'Ensemble'] == hp:
            df_final.loc[hp,'Ensemble'] = hp
            cont+=1
        else:
            cont+=1
            

x = 0
moda = 0
for i in HISTORIA_PREVIA: 
    metodo = []
    for j in colunas:
        if df_final.loc[i,j] == i:
            moda+= 1
            metodo.append(i)
    if(moda >= 3):
        for z in colunas:
            df_final.loc[i,z] = '1'
         
        else:
            df_final.loc[i,z] = '0'    
    else:
        for h in colunas:
            df_final.loc[i,h] = '0'
    x+=1
    moda=0

df_final

,Correlation,Backward Elimination,Recursive Feature Elimination,ANOVA,Embedded,Ensemble
Cancer,0,0,0,0,0,0
HAS,0,0,0,0,0,0
DM2,0,0,0,0,0,0
Cardiopatia Outra,0,0,0,0,0,0
Marcapasso,0,0,0,0,0,0
Sincope,0,0,0,0,0,0
Fibrilação/Flutter Atrial,0,0,0,0,0,0
I R Crônica,0,0,0,0,0,0
DLP,0,0,0,0,0,0
Coronariopatia,0,0,0,0,0,0


## 2. QUESTÃO DESAFIO
- Essa etapa não entrará com atividade avaliativa, pois a base de dados a seguir não é tão bonita quanto parece 🤪. Há muitas etapas de pré-processamento, agrupmaento e mais importante de tudo, o usuário tem que fazer as duas tabelas `weekly-infos-before-shrink.csv` e `user-status-after-shrink.csv` conversarem entre si.
- Fique à vontade para tentar e para tirar dúvidas. 😊

Nesse momento utilize a base de dados `weekly-infos-before-shrink.csv` e `user-status-after-shrink.csv`. Uma breve descrição da base:
- Essa base foi retirada de um serviço de streaming de midia. Os atributos são relativos a perfis de consumo de usuários.
- O objetivo é realizar detecção de *churn* (https://resultadosdigitais.com.br/blog/o-que-e-churn/)
- Possui um registro de 17 semanas de uso e os rótulos estáo na tabela `user-status-after-shrink.csv`, que indicam se ao final do período o usuário cancelou e se manteve assinante.

### a) Análise exploratória na base de dados
- Use a ferramenta `pandas-profiling` https://github.com/pandas-profiling/pandas-profiling
- Faça um resumo do relatório, destacando os principais pontos gerados na base.
> Exiba a matriz de correlação!

### b) Identicar qual a variáveis parecem influenciar no cancelamento da assinatura.

- Utilize o código `feature_selection_framework` https://github.com/navarmn/feature_selection_framework 
- Monte o resultado em um `DataFrame`, as colunas serão referentes aos métodos e as linhas deverão ser os atributos. Coloque 1 quando o método indicar como relevante e 0 quando o método indicar com não relevante;
- Utilize o seguinte critério para validar um atributo como relevante: moda >= 3

In [10]:
# code here